In [1]:
!pip install pygame

    100% |████████████████████████████████| 11.4MB 4.1MB/s eta 0:00:01
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel, DQN, STATE_ENCODING_LENGTH
from trainutil import train_models, test_models, get_epsilon_decay_factor
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
episodes = 150
steps = 1000 

gamma = 0.99

buffer_capacity = 15000
buffer_learn_thresh = 0.5
batch_size = 512

target_update = 10

start_epsilon = 0.99
min_epsilon = 0.05
decay_window = 100
decay = get_epsilon_decay_factor(start_epsilon, min_epsilon, decay_window)


calc_decay_window = episodes - int(buffer_learn_thresh * buffer_capacity / steps)
print("decay_window calc:", calc_decay_window)
print("Learning starts at episode:", int(buffer_learn_thresh * buffer_capacity / steps))
print("Decay rate: ", decay)

decay_window calc: 143
Learning starts at episode: 7
Decay rate:  0.9705844924924611


In [8]:
net = DQN(STATE_ENCODING_LENGTH, 8)
fs.load_net_from_device(net, "drl_fc32", "cpu")

DQN(
  (fc1): Linear(in_features=9, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=8, bias=True)
  (relu): ReLU()
)

In [4]:
env = GameState()
deep_rl_model = DeepRLModel(
                    epsilon = start_epsilon,
                    min_epsilon = min_epsilon,
                    epsilon_decay = decay,
                    gamma = gamma,
                    buffer_capacity = buffer_capacity,
                    replay_buffer_learn_thresh = buffer_learn_thresh,
                    batch_size = batch_size,
                    lr = 1e-3,
                    model = None)

# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=9, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=8, bias=True)
  (relu): ReLU()
)


In [ ]:
model_name = "drl_fc32_8"
train_models(env, models, episodes=episodes, steps=steps, target_update=target_update, print_every=1000, model_name=model_name)
# test_models(env, models, steps=2000)


TRAIN MODE
=== Starting Episode 0 ===


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean Episode Loss: nan | Episode Reward: 1.6683 | Mean Reward: 1.6683
=== Starting Episode 1 ===
Mean Episode Loss: nan | Episode Reward: -5.9162 | Mean Reward: -2.1240
=== Starting Episode 2 ===
Mean Episode Loss: nan | Episode Reward: -5.8944 | Mean Reward: -3.3808
=== Starting Episode 3 ===
Mean Episode Loss: nan | Episode Reward: -2.7780 | Mean Reward: -3.2301
=== Starting Episode 4 ===
Mean Episode Loss: nan | Episode Reward: -2.7493 | Mean Reward: -3.1339
=== Starting Episode 5 ===
Mean Episode Loss: nan | Episode Reward: -5.6270 | Mean Reward: -3.5494
=== Starting Episode 6 ===
Mean Episode Loss: nan | Episode Reward: 1.9080 | Mean Reward: -2.7698
=== Starting Episode 7 ===
----LEARNING BEGINS----
Mean Episode Loss: 0.0181 | Episode Reward: 2.4975 | Mean Reward: -2.1114
=== Starting Episode 8 ===
Mean Episode Loss: 0.0080 | Episode Reward: -2.0458 | Mean Reward: -2.1041
=== Starting Episode 9 ===
Mean Episode Loss: 0.0089 | Episode Reward: -1.4666 | Mean Reward: -2.0403
=== Star

Mean Episode Loss: 2.2489 | Episode Reward: 3.2049 | Mean Reward: 35.2317
=== Starting Episode 81 ===
Mean Episode Loss: 1.8959 | Episode Reward: 100.1832 | Mean Reward: 44.2890
=== Starting Episode 82 ===
Mean Episode Loss: 2.1134 | Episode Reward: 11.0122 | Mean Reward: 45.3785
=== Starting Episode 83 ===
Mean Episode Loss: 1.8964 | Episode Reward: 14.9116 | Mean Reward: 45.5844
=== Starting Episode 84 ===
Mean Episode Loss: 2.2237 | Episode Reward: -0.5257 | Mean Reward: 44.9016
=== Starting Episode 85 ===
Mean Episode Loss: 1.9321 | Episode Reward: 21.7082 | Mean Reward: 34.3731
=== Starting Episode 86 ===
Mean Episode Loss: 2.0551 | Episode Reward: 18.3965 | Mean Reward: 20.3842
=== Starting Episode 87 ===
Mean Episode Loss: 1.8820 | Episode Reward: 10.8707 | Mean Reward: 18.9050
=== Starting Episode 88 ===
Mean Episode Loss: 2.1314 | Episode Reward: 16.3464 | Mean Reward: 20.8015
=== Starting Episode 89 ===
Mean Episode Loss: 3.1067 | Episode Reward: 112.9585 | Mean Reward: 30.90

In [ ]:
fs.save_net_to_disk(deep_rl_model.model, model_name)